# Curl

This section demonstrates how to compute the curl of a vector field using UXarray.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import uxarray as ux

## Mathematical Background

The curl of a 2D vector field **V** = (u, v) is a scalar field that measures the rotation or circulation density at each point:

$$\text{curl}(\mathbf{V}) = \frac{\partial v}{\partial x} - \frac{\partial u}{\partial y}$$

Where:
- u is the zonal (x-direction) component of the vector field
- v is the meridional (y-direction) component of the vector field

The curl represents the tendency of the vector field to rotate around a point. Positive curl indicates counterclockwise rotation, while negative curl indicates clockwise rotation.

## Implementation

UXarray computes the curl using the existing gradient infrastructure:
1. Compute the gradient of the u-component: ∇u = (∂u/∂x, ∂u/∂y)
2. Compute the gradient of the v-component: ∇v = (∂v/∂x, ∂v/∂y)
3. Extract the cross-derivatives: ∂v/∂x and ∂u/∂y
4. Compute: curl = ∂v/∂x - ∂u/∂y

## Load Sample Data

We'll use a sample dataset to demonstrate curl computation.

In [ ]:
# Load a sample grid
grid = ux.open_grid(ux.grid.quad_hexagon())
uxds = ux.UxDataset({}, uxgrid=grid)

print(f"Grid dimensions: {uxds.uxgrid.n_face} faces")
print(f"Grid type: {type(uxds.uxgrid)}")

## Basic Curl Computation

Let's start with a simple example using coordinate-based vector fields.

In [ ]:
# Create vector field components using face coordinates
face_lon = uxds.uxgrid.face_lon
face_lat = uxds.uxgrid.face_lat

# Example 1: Linear vector field
u_linear = face_lon  # u = x
v_linear = face_lat  # v = y

# Compute curl
curl_linear = u_linear.curl(v_linear)

print(f"Curl shape: {curl_linear.shape}")
print(f"Curl range: [{curl_linear.min().values:.6f}, {curl_linear.max().values:.6f}]")
print(
    f"Finite values: {np.sum(np.isfinite(curl_linear.values))} / {len(curl_linear.values)}"
)

## Constant Vector Field

The curl of a constant vector field should be zero everywhere.

In [ ]:
# Create constant vector field
u_const = face_lat * 0 + 2.0  # u = 2 everywhere
v_const = face_lat * 0 + 3.0  # v = 3 everywhere

# Compute curl
curl_const = u_const.curl(v_const)

# Check that curl is approximately zero
finite_mask = np.isfinite(curl_const.values)
finite_curl = curl_const.values[finite_mask]

if len(finite_curl) > 0:
    max_abs_curl = np.abs(finite_curl).max()
    print(f"Maximum absolute curl of constant field: {max_abs_curl:.2e}")
    print(f"Mean curl: {finite_curl.mean():.2e}")
else:
    print("No finite curl values found")

## Rotational Vector Field

Let's create a vector field with pure rotation to see non-zero curl.

In [ ]:
# Create rotational vector field centered at grid center
center_lon = face_lon.mean()
center_lat = face_lat.mean()

# Rotational field: u = -y, v = x (counterclockwise rotation)
u_rot = -(face_lat - center_lat)  # u = -y
v_rot = face_lon - center_lon  # v = x

# Compute curl
curl_rot = u_rot.curl(v_rot)

# Analyze results
finite_mask = np.isfinite(curl_rot.values)
finite_curl = curl_rot.values[finite_mask]

if len(finite_curl) > 0:
    print("Curl statistics for rotational field:")
    print(f"  Mean: {finite_curl.mean():.6f}")
    print(f"  Std:  {finite_curl.std():.6f}")
    print(f"  Min:  {finite_curl.min():.6f}")
    print(f"  Max:  {finite_curl.max():.6f}")
    print("\nFor pure rotation u=-y, v=x, theoretical curl = 2")
else:
    print("No finite curl values found")

## Visualization

Let's visualize the vector field and its curl.

In [ ]:
# Create a figure with subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot u-component
u_rot.plot(ax=axes[0], cmap="RdBu_r")
axes[0].set_title("U-component (u = -y)")
axes[0].set_aspect("equal")

# Plot v-component
v_rot.plot(ax=axes[1], cmap="RdBu_r")
axes[1].set_title("V-component (v = x)")
axes[1].set_aspect("equal")

# Plot curl
curl_rot.plot(ax=axes[2], cmap="RdBu_r")
axes[2].set_title("Curl of Vector Field")
axes[2].set_aspect("equal")

plt.tight_layout()
plt.show()

## Advanced Operations

### Working with Real Data

In practice, you'll often work with velocity fields from atmospheric or oceanic models.

In [ ]:
# Example with synthetic atmospheric data
# Create a more complex vector field (e.g., representing wind)

# Simulate a vortex-like pattern
x = face_lon - center_lon
y = face_lat - center_lat
r = np.sqrt(x**2 + y**2)

# Avoid division by zero
r = np.maximum(r, 1e-10)

# Create a vortex with exponential decay
strength = np.exp(-(r**2) / 0.1)  # Gaussian decay
u_vortex = -y * strength / r**2  # Tangential component
v_vortex = x * strength / r**2

# Compute curl
curl_vortex = u_vortex.curl(v_vortex)

print("Vortex curl statistics:")
finite_mask = np.isfinite(curl_vortex.values)
finite_curl = curl_vortex.values[finite_mask]

if len(finite_curl) > 0:
    print(f"  Mean: {finite_curl.mean():.6f}")
    print(f"  Std:  {finite_curl.std():.6f}")
    print(f"  Range: [{finite_curl.min():.6f}, {finite_curl.max():.6f}]")

### Error Handling and Validation

UXarray provides comprehensive input validation for curl computation.

In [ ]:
# Example of error handling
try:
    # This will raise an error - wrong input type
    result = u_rot.curl(face_lon.values)  # numpy array instead of UxDataArray
except TypeError as e:
    print(f"TypeError caught: {e}")

try:
    # This will raise an error - different grids
    other_grid = ux.open_grid(ux.grid.quad_hexagon())
    other_data = ux.UxDataArray(
        np.ones(other_grid.n_face), dims=["n_face"], uxgrid=other_grid
    )
    result = u_rot.curl(other_data)
except ValueError:
    print("Value Error")

## Vector Calculus Identities

The curl operation satisfies several important mathematical identities.

In [ ]:
# Identity: curl of gradient is zero
# For any scalar field φ, curl(∇φ) = 0

# Create a scalar field
scalar_field = face_lon**2 + face_lat**2  # φ = x² + y²

# Compute its gradient
grad = scalar_field.gradient()
grad_u = grad["zonal_gradient"]
grad_v = grad["meridional_gradient"]

# Compute curl of gradient
curl_of_grad = grad_u.curl(grad_v)

# Check that it's approximately zero
finite_mask = np.isfinite(curl_of_grad.values)
finite_curl = curl_of_grad.values[finite_mask]

if len(finite_curl) > 0:
    max_abs_curl = np.abs(finite_curl).max()
    print(f"Maximum absolute curl of gradient: {max_abs_curl:.2e}")
    print("This should be close to zero (numerical precision limit)")

    # Check if it's within numerical precision
    if max_abs_curl < 1e-10:
        print("✓ Identity curl(∇φ) = 0 verified within numerical precision")
    else:
        print(f"⚠ Identity not perfectly satisfied (max error: {max_abs_curl:.2e})")
else:
    print("No finite values to check")

## Performance and Best Practices

### Tips for Efficient Curl Computation

1. **Data Preparation**: Ensure your vector components are on the same grid and have the same dimensions
2. **Memory Management**: For large datasets, consider processing subsets or using chunking
3. **Boundary Handling**: Curl values at boundary faces may be NaN due to incomplete stencils
4. **Validation**: Always check for finite values and validate against known analytical solutions when possible

In [ ]:
# Example of checking boundary effects
print(f"Total faces: {len(curl_rot.values)}")
print(f"Finite curl values: {np.sum(np.isfinite(curl_rot.values))}")
print(f"NaN values (likely boundaries): {np.sum(np.isnan(curl_rot.values))}")
print(f"Infinite values: {np.sum(np.isinf(curl_rot.values))}")

## Summary

This notebook demonstrated:

1. **Basic curl computation** using the `.curl()` method
2. **Mathematical validation** with constant and rotational fields
3. **Vector calculus identities** like curl(∇φ) = 0
4. **Practical examples** with synthetic atmospheric data
5. **Error handling** and input validation
6. **Performance considerations** and best practices

The curl operation is fundamental in fluid dynamics, electromagnetism, and many other fields. UXarray's implementation provides a robust and efficient way to compute curl on unstructured grids while maintaining mathematical accuracy and providing comprehensive error checking.